# Train And Test - Regression XPU

## Linear Regression

- Imports
  - standard libs
  - 3rd party libs
  - alpabetical or logical grouping
- Set random seed
- Config and Hyperparams
- Dataset and Dataloader
- Model definition/class
- Helper functions (training, eval, visualization)
- Then main code

Note: You can flip torch.amp on and off to test, this is work on XPU. Note this is not a great example case for leveraging amp but it is functional for testing. This is a setting with the hyperparameters.

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from pathlib import Path
from torch.amp import GradScaler, autocast

# Random seed number
RANDOM_SEED = 1

# Setup device agnostic code, you can extend to include cuda as well
# device = "xpu" if torch.xpu.is_available() else "cuda" if torch.cuda.is_available else "cpu"
# This will try to use xpu, fallback to cuda, then to cpu, I have this in my mixed gpu environments
device = "xpu" if torch.xpu.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize Hyperparameters
# Use step to increase or decrease the dataset size, lower number = more data, higher number = less data
start, end, step, weight, bias = 0, 1, 0.0002, 0.7, 0.3
learning_rate = 0.01
epochs = 1000
# Set to True to use mixed precision training (automatic mixed precision)
# This will be less accurate when turned on but technically faster
use_amp = True

# Subclass nn.Module to make our model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Use nn.Linear() for creating the model parameters
        self.linear_layer = nn.Linear(in_features=1, 
                                      out_features=1)
    
    # Define the forward computation (input data x flows through nn.Linear())
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

# Generate dataset and return the training and test data
def generate_dataset(start=0, end=1, step=0.0002, weight=0.7, bias=0.3):
    """
    Generate a dataset for a sample linear regression problem.

    Args:
        start (float): The start of the X values.
        end (float): The end of the X values.
        step (float): The step between X values.
        weight (float): The weight of the linear equation.
        bias (float): The bias of the linear equation.

    Returns:
        tuple: X_train, y_train, X_test, y_test
    """
    
    # Create X and y (features and labels)
    X = torch.arange(start, end, step).unsqueeze(dim=1) # without unsqueeze, errors will happen later on (shapes within linear layers)
    y = weight * X + bias 
    X[:10], y[:10]
    
    # Split data
    train_split = int(0.8 * len(X))
    X_train, y_train = X[:train_split], y[:train_split]
    X_test, y_test = X[train_split:], y[train_split:]

    # Return data
    return X_train, y_train, X_test, y_test

# Generate dataset and return the training and test data
X_train, y_train, X_test, y_test = generate_dataset(start, end, step, weight, bias)

# Plotting function to visualize the data
def plot_predictions(train_data=X_train.cpu(), 
                     train_labels=y_train.cpu(), 
                     test_data=X_test.cpu(), 
                     test_labels=y_test.cpu(), 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});



# Print out the shapes of our training and test datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Plot the training and test data
plot_predictions(X_train, y_train, X_test, y_test)

# Set the manual seed when creating the model (this isn't always needed but is used for demonstrative purposes, try commenting it out and seeing what happens)
torch.manual_seed(RANDOM_SEED)
torch.xpu.manual_seed(RANDOM_SEED)
model_0 = LinearRegressionModel()
#model_0, model_0.state_dict()

# Check model device
next(model_0.parameters()).device

# Set model to GPU if it's available, otherwise it'll default to CPU
model_0.to(device) # the device variable was set above to be "cuda" if available or "cpu" if not
next(model_0.parameters()).device

# Create loss function
loss_fn = nn.L1Loss()

# Create optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), # optimize newly created model's parameters
                            lr=learning_rate)

torch.manual_seed(RANDOM_SEED)
torch.xpu.manual_seed(RANDOM_SEED)

# Set the number of epochs 
epochs = 1000 

# Put data on the available device
# Without this, error will happen (not all model/data on device)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

if use_amp:
    # We'll use GradScaler to help with mixed precision training
    scaler = torch.amp.GradScaler(device)

for epoch in range(epochs):
    ### Training
    model_0.train() # train mode is on by default after construction

    if use_amp:
        # Forward pass with autocast for mixed precision
        with torch.amp.autocast(device):
            # 1. Forward pass
            y_pred = model_0(X_train)

            # 2. Calculate loss
            loss = loss_fn(y_pred, y_train)
    else:
        # 1. Forward pass
        y_pred = model_0(X_train)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y_train)

    # 3. Zero grad optimizer
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Step the optimizer
    optimizer.step()

    ### Testing
    model_0.eval() # put the model in evaluation mode for testing (inference)
    # 1. Forward pass
    with torch.inference_mode():
        # Forward pass with autocast for mixed precision
        
        if use_amp:
            with torch.amp.autocast(device):
                test_pred = model_0(X_test)
        else:
            test_pred = model_0(X_test)
    
        # 2. Calculate the loss
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")

# Find our model's learned parameters
from pprint import pprint # pprint = pretty print, see: https://docs.python.org/3/library/pprint.html 
print("The model learned the following values for weights and bias:")
pprint(model_0.state_dict())
print("\nAnd the original values for weights and bias are:")
print(f"weights: {weight}, bias: {bias}")

# Turn model into evaluation mode
model_0.eval()

# Make predictions on the test data
with torch.inference_mode():
    y_preds = model_0(X_test)

# Put data on the CPU and plot it
plot_predictions(predictions=y_preds.cpu())

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = "LinearRegressionModel_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 

# Instantiate a fresh instance of LinearRegressionModelV2
loaded_model_0 = LinearRegressionModel()

# Load model state dict 
loaded_model_0.load_state_dict(torch.load(MODEL_SAVE_PATH))

# Put model to target device (if your data is on GPU, model will have to be on GPU to make predictions)
loaded_model_0.to(device)

print(f"Loaded model:\n{loaded_model_0}")
print(f"Model on device:\n{next(loaded_model_0.parameters()).device}")

# Evaluate loaded model
loaded_model_0.eval()
with torch.inference_mode():
    loaded_model_0_preds = loaded_model_0(X_test)
y_preds == loaded_model_0_preds